# Salary Prediction in Mexico

## Importing

In [33]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import Lasso, Ridge

## Dataframes

### Dataframe from kaggle with target variable <=50k, >50k

In [34]:
dataset_path = ".\\Documents\\MachineLearningProject\\Dataset\\salary.csv"
df_50k = pd.read_csv(dataset_path)
df_50k.shape
df_50k.sample()


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
22815,33,Private,175412,9th,5,Divorced,Craft-repair,Unmarried,White,Male,114,0,55,United-States,<=50K


Filter the dataset to only have mexican data

In [35]:
df_50k = df_50k[df_50k["native-country"].str.contains("Mexico", case=False, na=False)]
df_50k.shape

(643, 15)

### Dataframes from the Mexican government

In [36]:
path = '.\\Documents\\MachineLearningProject\\Profesionistas y Tecnicos\\Evolucion-poblacion-ocupada-trabajadores-totales\\Evolucion-poblacion-ocupada-trabajadores-totales.csv'
professionals_df = pd.read_csv(path)
professionals_df.sample()
professionals_df.columns

Index(['Category ID', 'Category', 'Quarter ID', 'Quarter', 'Workforce',
       'Monthly Wage', 'Time', 'type', 'type ID'],
      dtype='object')

Filter unnecesary or redundance data.

In [37]:
columns_to_pop = ['type', 'type ID', 'Workforce', 'Time', 'Quarter']
for column in columns_to_pop:
    professionals_df.pop(column)
professionals_df

,Category ID,Category,Quarter ID,Monthly Wage
0,1,"Funcionarios, Directores y Jefes",20101,5652.376449
1,1,"Funcionarios, Directores y Jefes",20102,5176.334410
2,1,"Funcionarios, Directores y Jefes",20103,5528.734344
3,1,"Funcionarios, Directores y Jefes",20104,5201.022173
4,1,"Funcionarios, Directores y Jefes",20111,4678.875804
...,...,...,...,...
213,4,"Comerciantes, Empleados en Ventas y Agentes de...",20232,4363.811313
214,4,"Comerciantes, Empleados en Ventas y Agentes de...",20233,4502.061632
215,4,"Comerciantes, Empleados en Ventas y Agentes de...",20234,4617.991412
216,4,"Comerciantes, Empleados en Ventas y Agentes de...",20241,4925.742507


In [38]:
category_values = professionals_df['Category'].unique()
category_keys = professionals_df['Category ID'].unique()

category_dict = {}
for i in range(len(category_values)):
    category_dict[category_keys[i]] = category_values[i]

In [39]:
print(category_dict)

{1: 'Funcionarios, Directores y Jefes', 2: 'Profesionistas y Técnicos', 3: 'Trabajadores Auxiliares en Actividades Administrativas', 4: 'Comerciantes, Empleados en Ventas y Agentes de Ventas'}


So now we can pop the Category column.

In [40]:
professionals_df.pop('Category')
professionals_df.sample()

,Category ID,Quarter ID,Monthly Wage
17,1,20142,7728.990406


Dataframe salary per sex and scolar years

In [41]:
sex_and_education_paths = ['.\\Documents\\MachineLearningProject\\actividades administrativas\\Salario-segun-sexo-y-escolaridad-en-segundo-trimestre-de-2024-trabajadores-totales (2)\\Salario-segun-sexo-y-escolaridad-en-segundo-trimestre-de-2024-trabajadores-totales.csv', 
                           '.\\Documents\\MachineLearningProject\\Comerciantes y Venta\\Salario-segun-sexo-y-escolaridad-en-segundo-trimestre-de-2024-trabajadores-totales (1)\\Salario-segun-sexo-y-escolaridad-en-segundo-trimestre-de-2024-trabajadores-totales.csv',
                           '.\\Documents\\MachineLearningProject\\Funcionarios directores y jefes\\Salario-segun-sexo-y-escolaridad-en-segundo-trimestre-de-2024-trabajadores-totales (3)\\Salario-segun-sexo-y-escolaridad-en-segundo-trimestre-de-2024-trabajadores-totales.csv',
                           '.\\Documents\\MachineLearningProject\\Profesionistas y Tecnicos\\Salario-segun-sexo-y-escolaridad-en-segundo-trimestre-de-2024-trabajadores-totales\\Salario-segun-sexo-y-escolaridad-en-segundo-trimestre-de-2024-trabajadores-totales.csv'
                        ]

sex_and_education_dfs = []
for path in sex_and_education_paths:
    sex_and_education_dfs.append(pd.read_csv(path))

sex_and_education_df = pd.concat(sex_and_education_dfs, axis = 0)
sex_and_education_df.sample()

,Category ID,Category,Schooling Years Range ID,Schooling Years Range,Sex ID,Sex,Monthly Wage,Workforce,Number of Records,Workforce Total,percentage
0,2,Profesionistas y Técnicos,1,0 a 3 Años de Escolaridad,1,Hombre,7165.597506,50125,188,54585,91.829257


We notice in Sex ID 1 is for male and 2 for female.

Here we pop the unnecesary columns.

In [42]:
columns_to_pop = ['Category', 'Workforce', 'Number of Records', 'percentage', 'Sex', 'Workforce Total']
for column in columns_to_pop:
    sex_and_education_df.pop(column)

In [43]:
sex_and_education_df.sample()

,Category ID,Schooling Years Range ID,Schooling Years Range,Sex ID,Monthly Wage
1,4,1,0 a 3 Años de Escolaridad,2,2894.223471


I am going to determine the schooling years range keys.

In [44]:
schooling_years_range_values = sex_and_education_df['Schooling Years Range'].unique()

In [45]:
schooling_years_range_keys = sex_and_education_df['Schooling Years Range ID'].unique()

In [46]:
schooling_years_range = {}
for i in range(len(schooling_years_range_keys)):
    schooling_years_range[schooling_years_range_keys[i]] = schooling_years_range_values[i]

In [47]:
print(schooling_years_range)

{2: '4 a 6 Años de Escolaridad', 3: '7 a 9 Años de Escolaridad', 4: '10 a 12 Años de Escolaridad', 5: '13 a 15 Años de Escolaridad', 6: '16 a 18 Años de Escolaridad', 7: 'Más de 18 Años de Escolaridad', 1: '0 a 3 Años de Escolaridad'}


Now I can pop that column.

In [48]:
sex_and_education_df.pop('Schooling Years Range')
sex_and_education_df.sample()

,Category ID,Schooling Years Range ID,Sex ID,Monthly Wage
2,2,2,2,4057.628495


Salary per state dataframe.

In [49]:
salary_per_state_paths = ['.\\Documents\\MachineLearningProject\\actividades administrativas\\Salarios-en-2024-T2 (2)\\Salarios-en-2024-T2.csv',
                          '.\\Documents\\MachineLearningProject\\Comerciantes y Venta\\Salarios-en-2024-T2 (1)\\Salarios-en-2024-T2.csv',
                          '.\\Documents\\MachineLearningProject\\Funcionarios directores y jefes\\Salarios-en-2024-T2 (3)\\Salarios-en-2024-T2.csv',
                          '.\\Documents\\MachineLearningProject\\Profesionistas y Tecnicos\\Salarios-en-2024-T2\\Salarios-en-2024-T2.csv']

salary_per_state_dfs = []
for path in salary_per_state_paths:
    salary_per_state_dfs.append(pd.read_csv(path))

salary_per_state_df = pd.concat(salary_per_state_dfs, axis = 0)

In [50]:
salary_per_state_df.sample()

,Category ID,Category,State ID,State,Quarter ID,Quarter,Monthly Wage,Monthly Wage Growth,Monthly Wage Growth Value,lastSalary
26,1,"Officers, Directors and Heads",27,Tabasco,20242,2024-Q2,11771.545091,-0.231997,-3555.923029,15327.46812


Pop unnecesary columns.

In [51]:
columns_to_pop = ['Category', 'Monthly Wage Growth', 'lastSalary', 'Monthly Wage Growth Value', 'Quarter']
for column in columns_to_pop:
    salary_per_state_df.pop(column)

In [52]:
salary_per_state_df.sample()

,Category ID,State ID,State,Quarter ID,Monthly Wage
20,2,21,Puebla,20242,6562.939723


We are doing another dictionary for state and state id.

In [53]:
state_values = salary_per_state_df['State'].unique()

In [54]:
satate_keys = salary_per_state_df['State ID'].unique()

In [55]:
state_dict = {}
for i in range(len(satate_keys)):
    state_dict[satate_keys[i]] = state_values[i]

In [56]:
print(state_dict)

{1: 'Aguascalientes', 2: 'Baja California', 3: 'Baja California Sur', 4: 'Campeche', 5: 'Coahuila de Zaragoza', 6: 'Colima', 7: 'Chiapas', 8: 'Chihuahua', 9: 'Ciudad de México', 10: 'Durango', 11: 'Guanajuato', 12: 'Guerrero', 13: 'Hidalgo', 14: 'Jalisco', 15: 'Estado de México', 16: 'Michoacán de Ocampo', 17: 'Morelos', 18: 'Nayarit', 19: 'Nuevo León', 20: 'Oaxaca', 21: 'Puebla', 22: 'Querétaro', 23: 'Quintana Roo', 24: 'San Luis Potosí', 25: 'Sinaloa', 26: 'Sonora', 27: 'Tabasco', 28: 'Tamaulipas', 29: 'Tlaxcala', 30: 'Veracruz de Ignacio de la Llave', 31: 'Yucatán', 32: 'Zacatecas'}


Now we're popping the state column.

In [57]:
salary_per_state_df.pop('State')

0                      Aguascalientes
1                     Baja California
2                 Baja California Sur
3                            Campeche
4                Coahuila de Zaragoza
                   ...               
27                         Tamaulipas
28                           Tlaxcala
29    Veracruz de Ignacio de la Llave
30                            Yucatán
31                          Zacatecas
Name: State, Length: 128, dtype: object

Now we have this dataframes from the mexican government's page:

In [58]:
salary_per_state_df.sample(5)

,Category ID,State ID,Quarter ID,Monthly Wage
1,4,2,20242,5824.193958
9,2,10,20242,9631.272175
17,1,18,20242,17121.578980
24,1,25,20242,15374.076885
8,1,9,20242,14561.198425


In [59]:
professionals_df.sample(5)

,Category ID,Quarter ID,Monthly Wage
167,4,20113,2109.477352
149,3,20213,4918.646183
86,2,20172,5406.248519
60,2,20104,7829.068868
101,2,20212,6151.813519


In [60]:
sex_and_education_df.sample(5)

,Category ID,Schooling Years Range ID,Sex ID,Monthly Wage
9,1,6,2,11237.044526
5,4,3,2,3872.118698
11,4,6,2,5639.579510
7,3,5,2,6539.482893
9,3,6,2,6403.393616


With this ID dictionaries:

In [61]:
print(schooling_years_range)
print(state_dict)
print(category_dict)

{2: '4 a 6 Años de Escolaridad', 3: '7 a 9 Años de Escolaridad', 4: '10 a 12 Años de Escolaridad', 5: '13 a 15 Años de Escolaridad', 6: '16 a 18 Años de Escolaridad', 7: 'Más de 18 Años de Escolaridad', 1: '0 a 3 Años de Escolaridad'}
{1: 'Aguascalientes', 2: 'Baja California', 3: 'Baja California Sur', 4: 'Campeche', 5: 'Coahuila de Zaragoza', 6: 'Colima', 7: 'Chiapas', 8: 'Chihuahua', 9: 'Ciudad de México', 10: 'Durango', 11: 'Guanajuato', 12: 'Guerrero', 13: 'Hidalgo', 14: 'Jalisco', 15: 'Estado de México', 16: 'Michoacán de Ocampo', 17: 'Morelos', 18: 'Nayarit', 19: 'Nuevo León', 20: 'Oaxaca', 21: 'Puebla', 22: 'Querétaro', 23: 'Quintana Roo', 24: 'San Luis Potosí', 25: 'Sinaloa', 26: 'Sonora', 27: 'Tabasco', 28: 'Tamaulipas', 29: 'Tlaxcala', 30: 'Veracruz de Ignacio de la Llave', 31: 'Yucatán', 32: 'Zacatecas'}
{1: 'Funcionarios, Directores y Jefes', 2: 'Profesionistas y Técnicos', 3: 'Trabajadores Auxiliares en Actividades Administrativas', 4: 'Comerciantes, Empleados en Ventas y